In [ ]:
# Initialization

import math
from datetime import datetime
import openpyxl as opxl
from openpyxl.utils.dataframe import dataframe_to_rows
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import model as ty
import proc as pr
import matplotlib.ticker as mtick
import sklearn.metrics

from IPython.core.display import HTML

import heat_transfer as bht

import fluids as fds
import ht 

import general as gen

import os

import scipy.integrate as integrate

In [ ]:
# Crée un dictionnaire par et récupère les données du fichiers 'Inputs.xlsx' avec pd.read_excel
par = pr.create_par()

# Trouve le path pour le fichier d'outputs, crée un Excel avec openpyxl et les renvoie
pathout,wbo = pr.create_out()
sheet_o = wbo.active
sheet_o.title = "Inputs"
# Pour titre fichier outputs
moment = str(datetime.now().hour)+'-'+str(datetime.now().minute)+'-'+str(datetime.now().second)

# data = r'\220321_TUV_test_condi_output_outputtions.xlsx'
# data = r'\2205_TUV_test_conditions_V4.3.1.xlsx'
# data = r'\230104_TUV_test_conditions_V4.41_MPP_0flowrate.xlsx'
data = r'\230104_TUV_test_conditions_V4.41_MPP.xlsx'
# data = r'\condi_TSL.xlsx'
# data = r'\210914_TUV_test_conditions_SPRING.xlsx'

condi = pr.create_inputs_from_excel(data)

In [ ]:
ty.change_air_layer(par,0.000)

In [ ]:
par["coeff_h_top_free"] = 2.5
par["coeff_h_top_forced"] = 2.
par["coeff_h_back"] = 2.5

In [ ]:
df_par,df_res,X,list_df,list_df_historic = ty.simu_condi(par,condi)

df_res["S_loc"] = float(df_par.loc["W"])*float(df_par.loc["L_riser"])*df_res["S"]
df_res["-Q_fluid_back"] = -df_res["Q_fluid_back"]
# df_res["-Q_f01"] = -df_res["Q_f01"]

In [ ]:
# Process condi

condi_output = condi.copy()

dict = {'ta':'T_amb','U':'u','mdot':'m_dot','tin':'T_fluid_in','te':'T_fluid_out','tm':'T_m'}

column_headers = list(condi_output.columns.values)

for i in range(len(column_headers)):
    head = column_headers[i]
    if head in list(dict.keys()):
        condi_output.rename(columns = {head:dict[head]}, inplace = True)

condi_output['Gp'] = 0.*condi_output['G']
condi_output['T_fluid_in'] = condi_output['T_fluid_in']+273.15
condi_output['T_fluid_out'] = condi_output['T_fluid_out']+273.15
condi_output['T_amb'] = condi_output['T_amb']+273.15

condi_output = condi_output.reset_index()

condi_output = condi_output.drop(columns=["index","Date","UTC"])

condi_output = condi_output.astype('float64')


In [ ]:
SK_condi_output = ty.find_a_i(condi_output,par)
SK_df_res = ty.find_a_i(df_res,par)
pr.display_a_i(ty.find_a_i(condi_output,par))
pr.display_a_i(ty.find_a_i(df_res,par))

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%reload_ext autoreload

In [ ]:
pr.disp_html(df_res)
pr.display_a_i(X)

In [ ]:
mse = sklearn.metrics.mean_squared_error(df_res['Q_dot / A_G'],condi_output['Q_dot / A_G'])
print(math.sqrt(mse))

In [ ]:
print('RMSE en pourcentage de la valeur moyenne de puissance thermique =',round(math.sqrt(mse)/condi_output['Q_dot / A_G'].mean()*100,2),'%')

In [ ]:
max(abs(df_res['Q_dot / A_G']-condi_output['Q_dot / A_G']))/condi_output['Q_dot / A_G'].mean()

In [ ]:
df_res['Q_dot / A_G'].mean()

In [ ]:
-df_res.loc[df_res["u"]==u_list[i]]['-(T_m - T_a)']

In [ ]:
df_res = df_res.assign(SK_power=0.)

In [ ]:
for i in range(len(df_res)):
    df_res.loc[i,'SK_power'] = (1/float(df_par.loc["A_G"]))*pr.comp_power_coeff(SK_df_res[0],float(df_par.loc["A_G"]),df_res.loc[i,'T_fluid_out'],df_res.loc[i,'G'],df_res.loc[i,'Gp'],df_res.loc[i,'T_fluid_in'],df_res.loc[i,'T_amb'],df_res.loc[i,'u'])

In [ ]:
condi_output = condi_output.assign(SK_power=0.)

In [ ]:
condi_output = condi_output.assign(TUV_power=0.)

In [ ]:
for i in range(len(condi_output)):
    condi_output.loc[i,'SK_power'] = (1/float(df_par.loc["A_G"]))*pr.comp_power_coeff(SK_condi_output[0],float(df_par.loc["A_G"]),condi_output.loc[i,'T_fluid_out'],condi_output.loc[i,'G'],condi_output.loc[i,'Gp'],condi_output.loc[i,'T_fluid_in'],condi_output.loc[i,'T_amb'],condi_output.loc[i,'u'])

In [ ]:
TUV_coeff = [0.399,16.798,0,1.1416,0,0,0.0127,0,0]

In [ ]:
SK_condi_output

In [ ]:
pr.display_a_i(SK_condi_output)

In [ ]:
for i in range(len(condi_output)):
    condi_output.loc[i,'TUV_power'] = (1/float(df_par.loc["A_G"]))*pr.comp_power_coeff(TUV_coeff,float(df_par.loc["A_G"]),condi_output.loc[i,'T_fluid_out'],condi_output.loc[i,'G'],condi_output.loc[i,'Gp'],condi_output.loc[i,'T_fluid_in'],condi_output.loc[i,'T_amb'],condi_output.loc[i,'u'])

In [ ]:
df_res.loc[17,'T_fluid_out']

In [ ]:
pr.disp_html(df_res)

In [ ]:
condi_39 = condi_output.loc[condi_output["u"]==3.9].copy()

In [ ]:
Xbis = ty.find_a_i(condi_39,par)

In [ ]:
pr.display_a_i(Xbis)

In [ ]:
9.697+(3.9-3)*8.727

In [ ]:
0.212-(3.9-3)*-0.191

In [ ]:
intercept/1000

In [ ]:
slope

In [ ]:
# u_list = [0.7,2.1,3.9]
u_list = [0.7,2.1,3.9]

color_list = ['blue','red','green','orange','cyan','pink']

for i in range(len(u_list)):

    plt.scatter(-df_res.loc[df_res["u"]==u_list[i]]['-(T_m - T_a)'],df_res.loc[df_res["u"]==u_list[i]]['Q_dot / A_G'],marker="+",color=color_list[2*i],label='Model 1D - u = '+str(u_list[i])+' m/s')
    plt.scatter(condi_output.loc[condi_output["u"]==u_list[i]]['tm-ta'],condi_output.loc[condi_output["u"]==u_list[i]]['Q_dot / A_G'],marker='+',color=color_list[2*i+1],label='TÜV - u = '+str(u_list[i])+' m/s')

    plt.plot(-df_res.loc[df_res["u"]==u_list[i]]['-(T_m - T_a)'],df_res.loc[df_res["u"]==u_list[i]]['SK_power'])
    plt.plot(condi_output.loc[condi_output["u"]==u_list[i]]['tm-ta'],condi_output.loc[condi_output["u"]==u_list[i]]['SK_power'])

    plt.plot(condi_output.loc[condi_output["u"]==u_list[i]]['tm-ta'],condi_output.loc[condi_output["u"]==u_list[i]]['TUV_power'])

    # plt.plot(condi_39['tm-ta'], fitLine, c='r')

    # plt.plot(condi_39['tm-ta'], fitLine2, c='g')

plt.xlabel('Tm - Ta (K)')
plt.ylabel('Q_dot / A_G (W/m2 coll.)')

plt.grid()
plt.legend()
plt.show()


In [ ]:
from scipy import stats
#linregress() renvoie plusieurs variables de retour. On s'interessera 
# particulierement au slope et intercept
slope, intercept, r_value, p_value, std_err = stats.linregress(condi_39['tm-ta'], condi_39['Q_dot / A_G'])

In [ ]:
def predict(x):
   return slope * x + intercept

In [ ]:
def predictp(x):
    return -17.55 * x + 0.384*1000

In [ ]:
fitLine = predict(condi_39['tm-ta'])

In [ ]:
fitLine2 = predictp(condi_39['tm-ta'])

In [ ]:
pr.display_a_i(SK_condi_output)

In [ ]:
# Epaisseur d'ailette

par["Heta"] = 0.020
par["L_f0"] = 0.020
par["D"] = 0.014

k_list = [0.2,0.5,1,30,100,237]

Lf0_list = [0.0001,0.010,0.015,0.020,0.025,0.030,0.035,0.040,0.045,0.050]

lambd_list = [0.001,0.002,0.003,0.004,0.005,0.006,0.007,0.008,0.009,0.010]

for ki in range(len(k_list)):

    par["k_ail"] = k_list[ki]

    A0 = []
    A1 = []
    a3 = []

    f0_area = []

    for i in range(len(lambd_list)):
        # R_INTER = par["R_INTER"]
        # old_R_abs = par["lambd_abs"]/par["k_abs"]
        # par["lambd_abs"]=test_list[i]
        # new_R_abs = par["lambd_abs"]/par["k_abs"]
        # par["R_INTER"] = R_INTER - old_R_abs + new_R_abs
        

        # par["Heta"] = Lf0_list[i]
        # par["L_f0"] = Lf0_list[i]


        par["lambd_ail"] = lambd_list[i]

        par["N_ail"] = round((par["D"]+par["w_abs"])/(par["D"]+par["lambd_ail"]))
        par["N_f0"] = round(par["N_ail"]*par["L_riser"]/par["w_abs"])


        df_par,df_res,X,list_df,list_df_historic = ty.simu_condi(par,condi)
        A0.append(pr.A0_A1(X)[0])
        A1.append(pr.A0_A1(X)[1])
        a3.append(round(X[0][3],3))
        f0_area.append(float(df_par.loc["w_abs"])*float(df_par.loc["N_f0"])*float(df_par.loc["L_f0"]))

    plt.scatter(np.array(lambd_list)*1000,np.array(A1),marker="o",label="k_fin = "+str(par["k_ail"]))

plt.xlabel('Fin thickness (mm)')
plt.ylabel('A1 (1,3 m/s)')
plt.legend()
plt.show()


In [ ]:
# Epaisseur et longueur ailette

par["k_ail"] = 0.5
par["D"] = 0.014

# par["D"] = (par["w_abs"]-par["N_ail"]*par["lambd_ail"])/(par["N_ail"]-1)

Lf0_list = [0.0001,0.010,0.015,0.020,0.025,0.030,0.035,0.040,0.045,0.050]

lambd_list = [0.001,0.002,0.003,0.004,0.005,0.006,0.007,0.008,0.009,0.010]

for ki in range(len(lambd_list)):

    par["lambd_ail"] = lambd_list[ki]

    par["N_ail"] = round((par["D"]+par["w_abs"])/(par["D"]+par["lambd_ail"]))
    par["N_f0"] = round(par["N_ail"]*par["L_riser"])/par["w_abs"]

    A0 = []
    A1 = []
    a3 = []

    f0_area = []

    for i in range(len(Lf0_list)):
        # R_INTER = par["R_INTER"]
        # old_R_abs = par["lambd_abs"]/par["k_abs"]
        # par["lambd_abs"]=test_list[i]
        # new_R_abs = par["lambd_abs"]/par["k_abs"]
        # par["R_INTER"] = R_INTER - old_R_abs + new_R_abs
        

        par["Heta"] = Lf0_list[i]
        par["L_f0"] = Lf0_list[i]


        df_par,df_res,X,list_df,list_df_historic = ty.simu_condi(par,condi)
        A0.append(pr.A0_A1(X)[0])
        A1.append(pr.A0_A1(X)[1])
        a3.append(round(X[0][3],3))
        f0_area.append(float(df_par.loc["w_abs"])*float(df_par.loc["N_f0"])*float(df_par.loc["L_f0"]))

    plt.scatter(np.array(Lf0_list)*1000,np.array(A1),marker="o",label="lambd_fin = "+str(par["lambd_ail"]*1000)+" mm")

plt.xlabel('Fin length (mm)')
plt.ylabel('A1 (1,3 m/s)')
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.show()

In [ ]:
# Conductivité et longueur ailette

par["lambd_ail"] = 0.005
par["D"] = 0.014
par["N_ail"] = round((par["D"]+par["w_abs"])/(par["D"]+par["lambd_ail"]))
par["N_f0"] = round(par["N_ail"]*par["L_riser"])/par["w_abs"]
# par["D"] = (par["w_abs"]-par["N_ail"]*par["lambd_ail"])/(par["N_ail"]-1)
print(par["N_ail"])

k_list = [0.2,0.5,1,30,100,237]

Lf0_list = [0.0001,0.010,0.015,0.020,0.025,0.030,0.035,0.040,0.045,0.050]

lambd_list = [0.001,0.002,0.003,0.004,0.005,0.006,0.007,0.008,0.009,0.010]

for ki in range(len(k_list)):

    par["k_ail"] = k_list[ki]

    A0 = []
    A1 = []
    a3 = []

    f0_area = []

    for i in range(len(Lf0_list)):
        # R_INTER = par["R_INTER"]
        # old_R_abs = par["lambd_abs"]/par["k_abs"]
        # par["lambd_abs"]=test_list[i]
        # new_R_abs = par["lambd_abs"]/par["k_abs"]
        # par["R_INTER"] = R_INTER - old_R_abs + new_R_abs
        

        par["Heta"] = Lf0_list[i]
        par["L_f0"] = Lf0_list[i]


        df_par,df_res,X,list_df,list_df_historic = ty.simu_condi(par,condi)
        A0.append(pr.A0_A1(X)[0])
        A1.append(pr.A0_A1(X)[1])
        a3.append(round(X[0][3],3))
        f0_area.append(float(df_par.loc["w_abs"])*float(df_par.loc["N_f0"])*float(df_par.loc["L_f0"]))

    plt.scatter(np.array(Lf0_list)*1000,np.array(A1),marker="o",label="k_fin = "+str(par["k_ail"]))

plt.xlabel('Fin length (mm)')
plt.ylabel('A1 (1,3 m/s)')
plt.legend()
plt.show()

In [ ]:
areas = {"fs_he_area" : float(df_par.loc["A_G"]),"bs_he_area" : float(df_par.loc["w_abs"])*float(df_par.loc["L_abs"]),"f0_area" : float(df_par.loc["w_abs"])*float(df_par.loc["N_f0"])*float(df_par.loc["L_f0"])}

In [ ]:
areas

In [ ]:
backside_he_area

In [ ]:
# Air layer

test_list = np.linspace(0.001,0.002,30)



A0 = []
A1 = []
a1 = []
a3 = []

for i in range(len(test_list)):
    ty.change_air_layer(par,test_list[i])
    df_par,df_res,X,list_df,list_df_historic = ty.simu_condi(par,condi)
    A0.append(pr.A0_A1(X)[0])
    A1.append(pr.A0_A1(X)[1])
    a1.append(round(X[0][1],3))
    a3.append(round(X[0][3],3))

In [ ]:
plt.plot(test_list,A0)
plt.show()

In [ ]:
plt.scatter(np.array(test_list)*1000,np.array(f0_area)/float(df_par.loc["A_G"]),marker="o",color='blue')
plt.xlabel('Length of fins')
plt.ylabel('Fins area related to gross (m2/m2)')
plt.show()

In [ ]:
# Glass

test_list = np.linspace(0.001,0.010,20)

A0 = []
A1 = []

for i in range(len(test_list)):
    R_TOP = par["R_TOP"]
    old_R_glass = par["lambd_glass"]/par["k_glass"]
    par["lambd_glass"]=test_list[i]
    new_R_glass = par["lambd_glass"]/par["k_glass"]
    par["R_TOP"] = R_TOP - old_R_glass + new_R_glass
    df_par,df_res,X,list_df,list_df_historic = ty.simu_condi(par,condi)
    A0.append(pr.A0_A1(X)[0])
    A1.append(pr.A0_A1(X)[1])

In [ ]:
df_res["-Q_top_conv"]=-df_res["Q_top_conv"]
df_res["-Q_top_rad"]=-df_res["Q_top_rad"]

In [ ]:
display(HTML(df_par.to_html()))

In [ ]:
bht.Nu_forced_flat_plate_isoflux_lam(1.5, 0.024,3,15.6*10**(-6),0.707)

In [ ]:
(1/1.5)*integrate.quad(bht.Nu_forced_flat_plate_isoflux_lam,0,1.5,args=(0.024,3.,15.6*10**(-6),0.707))[0]

In [ ]:
bht.h_top_forced(300,290,2.7,1.5)

In [ ]:
1/(263*df_par.loc['p_tube'])+1/df_par.loc['C_B']

In [ ]:
1/(df_par.loc['p_tube']/(1/263+1/df_par.loc['C_B']))

In [ ]:
1/1.468

In [ ]:
300*(df_res["Q_fluid1"]-(df_res["Q_PV_plate"]-df_res["Q_fluid_back"]-df_res["Q_f01"]))

In [ ]:
float(df_par.loc["W"])*float(df_par.loc["L_riser"])*df_res["S"]

In [ ]:
(float(df_par.loc["W"])*float(df_par.loc["L_riser"])*df_res["S"]-df_res["Q_top_conv"]-df_res["Q_top_rad"])-df_res["Q_PV_plate"]

In [ ]:
((((float(df_par.loc["W"])*float(df_par.loc["L_riser"])*df_res["S"]-df_res["Q_top_conv"]-df_res["Q_top_rad"])-df_res["Q_fluid_back"]-df_res["Q_f01"])-df_res["Q_fluid1"])*300)/(df_res["Q_fluid1"]*300)

In [ ]:
df_select_u = df_res.loc[df_res["u"]<=1.]

In [ ]:
display(HTML(df_select_u.to_html()))

In [ ]:
df_hist = df_select_u

dict_pos = {}
dict_neg = {}
dict_pos_agg = {}
dict_neg_agg = {}

DT_list = []
for DT in df_hist["-(T_m - T_a)"]:
    DT_list.append(-DT)

powers = ["S_loc","-Q_top_conv","-Q_top_rad","-Q_fluid_back"]
# powers = ["S_loc","-Q_top_conv","-Q_top_rad","-Q_fluid_back","-Q_f01"]


for i in range(len(powers)):
    str = powers[i]
    
    dict_pos[str] = 300*np.array(ty.pos_df(df_hist,str))
    dict_neg[str] = 300*np.array(ty.neg_df(df_hist,str))

    if i==0:
        dict_pos_agg[str] = 0*np.array(ty.pos_df(df_hist,str))
        dict_neg_agg[str] = dict_pos_agg[str]

    else:
        dict_pos_agg[str] = dict_pos_agg[powers[i-1]] + dict_pos[powers[i-1]]
        dict_neg_agg[str] = dict_neg_agg[powers[i-1]] + dict_neg[powers[i-1]]

In [ ]:
dict_pos["S_loc"]

In [ ]:
fig, ax = plt.subplots()

width = 0.35

for str in powers:
    ax.bar(DT_list, dict_pos[str], width, bottom=dict_pos_agg[str], label=str)
    ax.bar(DT_list, dict_neg[str], width, bottom=dict_neg_agg[str], label=str)

ax.scatter(DT_list,300*df_hist["Q_fluid1"],color="red",label="Q_fluid")
# ax.plot(labels,np.array(N_disp*[list_PL[q]]))

# ax.set_ylabel('PL (Pa)')
# ax.set_title('SPRING')
ax.set_xlabel("Tm - Ta (K)")
ax.set_ylabel("Power (W)")
ax.axhline(y = 0., color = 'g', linestyle = '-')
  
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))

plt.show()

In [ ]:
df_res["Q_PV_plate"]+df_res["Q_top_conv"]+df_res["Q_top_rad"]

In [ ]:
tab_mat

In [ ]:
pr.display

In [ ]:
X[0][len(X[0])-1]

In [ ]:
# Test complet TUV ou général (key word: wing)

### Creating Excel outputs

if par["test"] == "TUV":
    prefixe = r'\OutputsTUV'
else: # general
    prefixe = r'\Parametric'

suffixe = prefixe+str(par["ailette"])+r'-'+moment+r'.xlsx'
fichier_o = pathout+suffixe

for d in dataframe_to_rows(df_par, index=True, header=False):
    sheet_o.append(d)

wso = wbo.create_sheet("Outputs")

for d in dataframe_to_rows(df, index=True, header=True):
    wso.append(d)

huit = len(X[0])-1

index_coeff = ['a1','a2','a3','a4','a6','a7','a8']
ws_lsm = wbo.create_sheet("LSM")
ws_lsm['A'+str(1)] = 'a0'
ws_lsm['B'+str(1)] = X[0][len(X[0])-1]

print('a0 : ',round(X[0][len(X[0])-1],3))

ws_lsm['C'+str(1)] = X[3][len(X[0])-1]
for l in range(huit):
    ws_lsm['A'+str(l+2)] = index_coeff[l]
    if l != 3:
        ws_lsm['B'+str(l+2)] = -X[0][l]
        print(index_coeff[l],' : ',round(-X[0][l],3))

    else:
        ws_lsm['B'+str(l+2)] = X[0][l]
        print(index_coeff[l],' : ',round(X[0][l],3))

    ws_lsm['C'+str(l+2)] = X[3][l]

ws_lsm['A'+str(huit+2)] = 'a0 (DS) at 1,3 m/s'
ws_lsm['B'+str(huit+2)] = X[0][len(X[0])-1] - (-X[0][4]*(1.3-3))
print(round(X[0][len(X[0])-1] - (-X[0][4]*(1.3-3)),3)*100,'%')

ws_lsm['A'+str(huit+3)] = 'a1 (DS) at 1,3 m/s'
ws_lsm['B'+str(huit+3)] = -X[0][0] - X[0][2]*(1.3-3)
print(round(-X[0][0] - X[0][2]*(1.3-3),1))
    
wbo.save(filename = fichier_o)
wbo.close()

### Excel is created

print(df)

print("Finished")

In [ ]:
al = pd.DataFrame([[1,2,3],[2,3,4],[4,5,6]],columns=['a','b','c'])

In [ ]:
al.keys()[0]

In [ ]:
df

In [ ]:
dfbis = df.loc[df['u']==0.7]

In [ ]:
dfbis

In [ ]:
# Parametric study

N_list = np.linspace(40,120,10)

test_list = []
test_list = N_list

# for i in range(len(D_list)):
#     test_list.append(math.floor((D_list[i]+par["w_abs"])/(D_list[i]+par["lambd_ail"])))


# test_list = np.linspace(25,125,20)

A0 = []
A1 = []
A2 = []
A3 = []
A4 = []
A6 = []
A7 = []
A8 = []

for i in range(len(test_list)):

    pr.proc(par,par["test"],i,test_list)

    # Model
    # if 'TUV' in par["test"]:
    #     df_par,df,X = ty.simu_condi(par,condi)
    # else:
    #     df_par,df,X = ty.simu_multi_condi(par,G_list,coeff_Gp_list,T_amb_list,u_list,T_guess_list,T_f_in_list)

    df_par,df,X = ty.simu_condi(par,condi)

    A0.append(X[0][len(X[0])-1])
    A1.append(-X[0][0])
    A2.append(-X[0][1])
    A3.append(-X[0][2])
    A4.append(X[0][3])
    A6.append(-X[0][4])
    A7.append(-X[0][5])
    A8.append(-X[0][6])

# Creating Excel outputs

suffixe = r'\Parametric'+str(par["ailette"])+r'-'+moment+r'.xlsx'
fichier_o = pathout+suffixe

for d in dataframe_to_rows(df_par, index=True, header=False):
    sheet_o.append(d)

wso = wbo.create_sheet("Outputs")

wso['A'+str(1)] = par["test"]
wso['B'+str(1)] = 'a0'
wso['C'+str(1)] = 'a1'
wso['D'+str(1)] = 'a2'
wso['E'+str(1)] = 'a3'
wso['F'+str(1)] = 'a4'
wso['G'+str(1)] = 'a6'
wso['H'+str(1)] = 'a7'
wso['I'+str(1)] = 'a8'
wso['J'+str(1)] = 'a0 DS'
wso['K'+str(1)] = 'a1 DS'

A0DS = []
A1DS = []

for l in range(len(test_list)):
    wso['A'+str(l+2)] = test_list[l]
    wso['B'+str(l+2)] = A0[l]
    wso['C'+str(l+2)] = A1[l]
    wso['D'+str(l+2)] = A2[l]
    wso['E'+str(l+2)] = A3[l]
    wso['F'+str(l+2)] = A4[l]
    wso['G'+str(l+2)] = A6[l]
    wso['H'+str(l+2)] = A7[l]
    wso['I'+str(l+2)] = A8[l]
    wso['J'+str(l+2)] = A0[l]-A6[l]*(-1.7)
    wso['K'+str(l+2)] = A1[l]+A3[l]*(-1.7)

    A0DS.append(A0[l]-A6[l]*(-1.7))
    A1DS.append(A1[l]+A3[l]*(-1.7))

wbo.save(filename = fichier_o)
wbo.close()

print("Finished")

In [ ]:
print(test_list)

In [ ]:
# Plot parametric study

# create figure and axis objects with subplots()
fig,ax = plt.subplots()
# make a plot

ax.plot(np.array(test_list)*1000,A0DS,
        color="red", 
       marker="o")

# set x-axis label

ax.set_xlabel("Fin spacing (mm)", fontsize = 14)

# set y-axis label

ax.set_ylabel("A0 at u = 1.3 m/s",
              color="red",
              fontsize=14)

ax.set_ybound(0.4,0.6)

# ax.yaxis.set_major_formatter(mtick.PercentFormatter(xmax=1.0))
# ax.yaxis.set_major_formatter(mtick.FormatStrFormatter('%0.1f'))
ax.yaxis.set_major_formatter(mtick.FormatStrFormatter('%.2f'))

# ax.axhline(y = 0.999*0.396177,label='98% of the maximum performance',color='green')
# ax.axvline(x = 0.4, label = 'Thickness of the tested prototype',color='black')
# ax.axhline(y = 0.394,color='black')
# ax.axvline(x = 1.3,color='green')
# ax.legend()

# twin object for two dif       ferent y-axis on the sample plot
ax2=ax.twinx()
# make a plot with different y-axis using second axis object
ax2.plot(np.array(test_list)*1000,A1DS,color="blue",marker="o")
ax2.set_ylabel("A1 at u = 1.3 m/s",color="blue",fontsize=14)

# ax2.set_ybound(12.5,16)

# save the plot as a file
plt.savefig('parametric.png',format='png',dpi=200,bbox_inches='tight')

plt.show()

In [ ]:
a = {'a' : 1, 'b' : 2}
b = {'a' : 2, 'b' : 4}
l = [a,b]
dfl = pd.DataFrame(l)
dfm = dfl.mean()
print(dfl)
print(dfm)
print(dfm['a'])

dfh = pd.DataFrame([dfm,dfm,dfm])
print(dfh)

bonjour = 'a'
print(dfh[bonjour])

In [ ]:
print(test_list)
print(A0DS[5])

In [ ]:
np.max(A0DS)

In [ ]:
par["h_fluid"]

In [ ]:
# Test d'un panneau dans un set de conditions donné (résultats en détails tranche par tranche)

suffixe = r'\OnePanel-'+moment+r'.xlsx'
fichier_o = pathout+suffixe
var = {}

list_var,list_var_conv = ty.simu_one_steady_state(par,var,par["N_meander"],par["T_fluid_in0"],315,"all")

compt = 2

*var_names0, = var
var_names = ['T_fluid_in'] + var_names0

for m in range(len(var_names)):
    sheet_o.cell(row=1,column=m+1,value = var_names[m])

for l in range(par["N_meander"]):
    for m in range(len(var_names)):
        sheet_o.cell(row=l+2,column=m+1,value = list_var[l][var_names[m]])

sheet_o3 = wbo.create_sheet("Convergence")

var_names2 = ['Slice','T_fluid_in'] + var_names0

for m in range(len(var_names2)):
    sheet_o3.cell(row=1,column=m+1,value = var_names2[m])

for l in range(len(list_var_conv)):
    for m in range(len(var_names2)):
        sheet_o3.cell(row=l+2,column=m+1,value = list_var_conv[l][var_names2[m]])


sheet_o2 = wbo.create_sheet("T_abs")

x_list = np.linspace(0,par["delta"],100)

T_abs = []

delta = par["delta"]

for k in range(par["N_meander"]):
    b = list_var[k]["b"]
    j = list_var[k]["j"]
    m = list_var[k]["m"]
    T_B = list_var[k]["T_Base_mean"]

    b_j = b/j

    for l in range(len(x_list)):
        res = b_j+((T_B-b_j)/math.cosh(m*delta))*math.cosh(m*x_list[l])

        sheet_o2.cell(row=l+1,column=k+2,value = res)

for l in range(len(x_list)):
    sheet_o2.cell(row=l+1,column=1,value=x_list[l])

wbo.save(filename = fichier_o)
wbo.close()

print("Finished")

In [ ]:
hein = {'ah':1,'bh':2}
print(hein.values())
print(hein.keys())
hh = pd.DataFrame.from_dict({'row' : hein.values()},orient='index',columns=hein.keys())

In [ ]:
"main" in ["mai","bonjour"]

In [ ]:
yoo = pd.DataFrame()

In [ ]:
yoo['salut'] = [2]

In [ ]:
yoo